# Task C: Perform lift and sentiment analysis on each candidate and issue

### Extract data

In [1]:
import json
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from functools import reduce
from collections import Counter

In [2]:
def find_key_in_dict(keys_i_want, dictionary):
    l = []
    def iterate_through_dict(keys_i_want, dictionary):
        for i in dictionary:
            if i in keys_i_want:
                l.append({i: dictionary[i]})
                keys_i_want.remove(i)
            if type(dictionary[i]) is dict:
                iterate_through_dict(keys_i_want, dictionary[i])
    iterate_through_dict(keys_i_want,dictionary)            
    return l

In [3]:
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [4]:
with open('Oct14thTweetMine.json','r') as f:
#     print(sum([1 for i in f]))
    comments =[]
    #big_city = ['Dallas', 'Houston', 'San Antonio', 'Austin', 'El Paso']
    count = 1
    for line in f:
        tweet = json.loads(line)
        if count <= 4000:
            l = find_key_in_dict(['full_text', 'text'], tweet)  ####### I modified this(and following) lines to not collect location info. 
            d = {}
            for i in range(len(l)):
#                 for j in l[i]:
#                     if j == 'location':
#                         d['location'] = l[i][j]
#                         break
                for j in l[i]:
                    if j == 'full_text':
                        d['text'] = l[i][j]
                        break
                if 'text' not in d.keys():
                    for j in l[i]:
                        if j == 'text':
                            d['text'] = l[i][j]
                            break
#             print(d['text'])
            comments.append(d)
#             print(count,d)
#             print()
        count += 1
    
comments

[{'text': '"Ted Cruz has insisted that we repeal every single word of the Affordable Care Act, including protections for pre-existing conditions. I want to expand medicaid."\n\n— Beto O\'Rourke talks differences between himself and Ted Cruz in the Texas Senate race. https://t.co/Ko7Y8ttM2I'},
 {'text': 'What the hell is wrong with Texas? Republicans have one of the smarmiest men ever as their candidate for Senate, Ted Cruz. Democrats have one of the best candidates in a lifetime, Beto O’Rourke. And yet Cruz is up by eight points.\n#Beto4Texas'},
 {'text': '🛑Beto skates across the Stage on his skateboard. Hoping to show his “Cool Guy” side.  Cruz still leads in polling.Dont become complacent #Texas He’s raised $36 million to win. @realDonaldTrump https://t.co/dzlvM7GpIK🛑 https://t.co/MQQhmi076T'},
 {'text': '"Ted Cruz has insisted that we repeal every single word of the Affordable Care Act, including protections for pre-existing conditions. I want to expand medicaid."\n\n— Beto O\'Rourk

### Make all commends in to lower case for later analysis.

In [216]:
commentsdict = {}
commentcount = 1
for i in comments:
    for key in i:
        list1 = []
        list1.append(i[key].lower())
#         list1.append(clean_tweet(i[key]).lower())  #removing links and special characters 
    commentsdict['text' + str(commentcount)] = ' '.join(list1)
    commentcount += 1
print(commentsdict)

{'text1': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i', 'text2': 'what the hell is wrong with texas? republicans have one of the smarmiest men ever as their candidate for senate, ted cruz. democrats have one of the best candidates in a lifetime, beto o’rourke. and yet cruz is up by eight points.\n#beto4texas', 'text3': '🛑beto skates across the stage on his skateboard. hoping to show his “cool guy” side.  cruz still leads in polling.dont become complacent #texas he’s raised $36 million to win. @realdonaldtrump https://t.co/dzlvm7gpik🛑 https://t.co/mqqhmi076t', 'text4': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talk

### To collect four issues
#### For healthcare issue:
When people mention 'medicaid', 'health', we assume that they are talking about healthcare related issue.
#### For the rest of issues:
We collect both singular and dual type of the word that mention in the commends.

In [217]:
issues = {
    'healthcare': dict(),
    'job': dict(),
    'crime': dict(),
    'gun': dict(),
    }

In [218]:
for i in commentsdict:
    for j in word_tokenize(commentsdict[i]):
        if j == 'medicaid' or j == 'health':
            issues['healthcare'][i] = commentsdict[i]
        elif j == 'job' or j == 'jobs':
            issues['job'][i] = commentsdict[i]
        elif j == 'crime' or j == 'crimes':
            issues['crime'][i] = commentsdict[i]
        elif j == 'gun' or j == 'guns':
            issues['gun'][i] = commentsdict[i]

print(issues)

{'healthcare': {'text1': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i', 'text4': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i', 'text6': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i', 'text16': '"ted cruz has insisted that we repeal every single word of the affordable care act, including pro

In [219]:
issues_mention_list = []
for i in issues:
    issues_mention_list.append( (i, len(issues[i])))
issues_mention_list.sort(key=lambda x: x[1], reverse=True)
# print(len(issues_mention_list))
for i in range(len(issues_mention_list)):
    print(str(i+1) + ") The issue '" + str(issues_mention_list[i][0],) + "' was mentioned " + str(issues_mention_list[i][1]) + ' times')
        

1) The issue 'healthcare' was mentioned 87 times
2) The issue 'gun' was mentioned 3 times
3) The issue 'job' was mentioned 1 times
4) The issue 'crime' was mentioned 1 times


### To collect the candidates
#### For Beto O'Rourke:
We use the nicknames, beto, o'rourke, o'rourge, and betoorourke, as references to collect the commends. 
#### For Ted Cruz:
We use the nicknames, cruz and ted, as references to collect the commends.

In [220]:
candidates = {'beto':dict(), 'cruz':dict()}

for i in commentsdict:
    for j in word_tokenize(commentsdict[i]):
        if j == 'beto' or j == "o'rourke" or j == "o'rourge" or j == "betoorourke":
            candidates['beto'][i] = commentsdict[i]
        elif j == 'cruz' or j == 'ted':
            candidates['cruz'][i] = commentsdict[i]

print(candidates)

{'beto': {'text1': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i', 'text2': 'what the hell is wrong with texas? republicans have one of the smarmiest men ever as their candidate for senate, ted cruz. democrats have one of the best candidates in a lifetime, beto o’rourke. and yet cruz is up by eight points.\n#beto4texas', 'text4': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i', 'text5': 'what the hell is wrong with texas? republicans have one of the smarmiest men ever as their candidate for senate, ted cruz. democrats have on

In [221]:
candidates_mention_list = []
for i in candidates:
    candidates_mention_list.append( (i, len(candidates[i])))
candidates_mention_list.sort(key=lambda x: x[1], reverse=True)
# print(len(issues_mention_list))
for i in range(len(candidates_mention_list)):
    print(str(i+1) + ") The candidates '" + str(candidates_mention_list[i][0],) + "' was mentioned " + str(candidates_mention_list[i][1]) + ' times')
        

1) The candidates 'cruz' was mentioned 629 times
2) The candidates 'beto' was mentioned 531 times


### Collect the commends that people mentioned both one of the issues and one of the candidates.

In [222]:
cross_candidates_issues = dict()

# init dictionary
for i in issues:
    cross_candidates_issues[i] = dict()
    for j in candidates:
        cross_candidates_issues[i][j] = dict() 
print(cross_candidates_issues)
        
for i in candidates:
    for j in issues:
        for k in issues[j]:
            for l in word_tokenize(issues[j][k]):
                if l == i:
                    cross_candidates_issues[j][i][k] = issues[j][k]
                    break
cross_candidates_issues

{'healthcare': {'beto': {}, 'cruz': {}}, 'job': {'beto': {}, 'cruz': {}}, 'crime': {'beto': {}, 'cruz': {}}, 'gun': {'beto': {}, 'cruz': {}}}


{'healthcare': {'beto': {'text1': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i',
   'text4': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i',
   'text6': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i',
   'text16': '"ted cruz has insisted that we repeal every single word of the affordable care 

### Calculate the lift.

In [223]:
# now we can calculate the lift value
data=[]
for i in range(len(cross_candidates_issues.keys())):
    row = []
    for j in range(len(cross_candidates_issues[list(cross_candidates_issues.keys())[i]].keys())):
            row.append(round( len(cross_candidates_issues[list(issues.keys())[i]][list(candidates.keys())[j]])\
                             / (len(issues[list(issues.keys())[i]])\
                            * len(candidates[list(candidates.keys())[j]]) ) * len(commentsdict), 4))
    data.append(row)
data

[[1.5637, 1.3201], [0.0, 0.0], [0.0, 0.0], [1.0546, 0.4452]]

In [224]:
import pandas as pd
import numpy as np
lift_computed = []
for i in data:
    for j in i:
        lift_computed.append(j)
lift_computed
indexes = []
for i in issues.keys():
    for j in candidates.keys():
        a = str(i+' & '+j)
        indexes.append(a)
indexes
lift_chart = pd.DataFrame(lift_computed, columns=["Lift"], index=indexes)
lift_chart

,Lift
healthcare & beto,1.5637
healthcare & cruz,1.3201
job & beto,0.0000
job & cruz,0.0000
crime & beto,0.0000
crime & cruz,0.0000
gun & beto,1.0546
gun & cruz,0.4452


### Compute Sentiment Score

In [225]:
#create issues and candidates list which the lift of their combination are higher than 1 
issues1 = []
candidates1 = []
for i in range(len(cross_candidates_issues.keys())):
    for j in range(len(cross_candidates_issues[list(cross_candidates_issues.keys())[i]].keys())):
        lift = round( len(cross_candidates_issues[list(issues.keys())[i]][list(candidates.keys())[j]])\
                    / (len(issues[list(issues.keys())[i]])\
                    * len(candidates[list(candidates.keys())[j]]) ) * len(commentsdict), 4)
        if lift > 1:
            issues1.append(list(issues.keys())[i])
            candidates1.append(list(candidates.keys())[j])
    
issues1,candidates1

(['healthcare', 'healthcare', 'gun'], ['beto', 'cruz', 'beto'])

In [226]:
cross_lift_high1 = {}
# init dictionary for storing 'lift > 1' of cross_candidates_issues only.
for c in range(len(issues1)):
    for i in issues:
        if i in cross_lift_high1.keys():
            for j in candidates:
                if i == issues1[c] and j == candidates1[c]:
                    cross_lift_high1[i][j]= dict()
        elif i == issues1[c]:
            cross_lift_high1[i]= dict()
            for j in candidates:
                if i == issues1[c] and j == candidates1[c]:
                    cross_lift_high1[i][j]= dict()
print(cross_lift_high1)

count = 0
for c in range(len(issues1)):
    for i in cross_candidates_issues: #i  healthcare
        for j in cross_candidates_issues[i]: #j  beto
            if i == issues1[c] and j == candidates1[c]:
                for k in cross_candidates_issues[i][j]:
                    cross_lift_high1[i][j][k]= cross_candidates_issues[i][j][k]
                
cross_lift_high1

{'healthcare': {'beto': {}, 'cruz': {}}, 'gun': {'beto': {}}}


{'healthcare': {'beto': {'text1': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i',
   'text4': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i',
   'text6': '"ted cruz has insisted that we repeal every single word of the affordable care act, including protections for pre-existing conditions. i want to expand medicaid."\n\n— beto o\'rourke talks differences between himself and ted cruz in the texas senate race. https://t.co/ko7y8ttm2i',
   'text16': '"ted cruz has insisted that we repeal every single word of the affordable care 

### Import the library

In [227]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def cal_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return snt['compound']

### Create a function for extracting sentence that contains people's ememotions expressed with a close proximity of an attribute. We use 6 words as the radius.

In [234]:
from nltk import ngrams
def get_radius_list(sentence,str1):
#     sentence = 'new poll finds beto trailing 5but leading cruz by 60 points among12 voters who say health care is their top issue this is why beto 014has gone all in on xxdefendingxx obamacare and calling for medicaid expansion'
    long = len(sentence.split())
    # limit = int(((long-n)/n)+n)  
    limit = long-(n-1) # word long = 37 , pick 12 words every time. 15th time is the last time.
    
    sixgrams = ngrams(sentence.split(), 13)
    count= 1
    temp = []
#     str1 = 'expansion'

    for grams in sixgrams:
        if count == 1:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 2:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 3:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 4:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 5:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 6:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == 7:
            if grams[0] == str1:
                c = ' '.join(grams)
                temp.append(c)
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count < limit:
            if grams[6] == str1:
                c = ' '.join(grams)
                temp.append(c)
        elif count == limit:
            for i in range(6,13):
                if grams[i] == str1:
                    c = ' '.join(grams)
                    temp.append(c)
        count += 1
    return temp

In [235]:
cross_lift_score = dict()

# init dictionary to store the Sentiment Score
for i in issues:
    cross_lift_score[i] = dict()
    for j in candidates:
        cross_lift_score[i][j] = None 
print(cross_lift_score)

{'healthcare': {'beto': None, 'cruz': None}, 'job': {'beto': None, 'cruz': None}, 'crime': {'beto': None, 'cruz': None}, 'gun': {'beto': None, 'cruz': None}}


### Computing the Sentiment Score and storing the score in `cross_lift_score` dictionary.

In [236]:
total = 0
for i in cross_lift_high1: #healthcare
    if i == 'healthcare':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(cross_lift_high1[i][j][k]):
                    if w == 'medicaid' or w == 'health':
                        for l in get_radius_list(cross_lift_high1[i][j][k],w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    if i == 'job':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(cross_lift_high1[i][j][k]):
                    if w == 'job' or w == 'jobs':
                        for l in get_radius_list(cross_lift_high1[i][j][k],w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    if i == 'crime':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(cross_lift_high1[i][j][k]):
                    if w == 'crime' or w == 'crimes':
                        for l in get_radius_list(cross_lift_high1[i][j][k],w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
    if i == 'gun':
        for j in cross_lift_high1[i]: #beto
            for k in cross_lift_high1[i][j]: #text
                for w in word_tokenize(cross_lift_high1[i][j][k]):
                    if w == 'gun' or w == 'guns':
                        for l in get_radius_list(cross_lift_high1[i][j][k],w):
                            total += cal_sentiment_scores(l)
                            avg = total/len(cross_lift_high1[i][j])
                            cross_lift_score[i][j]=avg
cross_lift_score

{'healthcare': {'beto': 0.00712093023255814, 'cruz': 0.01424186046511628},
 'job': {'beto': None, 'cruz': None},
 'crime': {'beto': None, 'cruz': None},
 'gun': {'beto': 0.0129, 'cruz': None}}

In [237]:
# Extracting the score from dictionary into list.
total_score = []
for i in cross_lift_score:
    for j in cross_lift_score[i]:
        total_score.append(cross_lift_score[i][j])
total_score

[0.00712093023255814,
 0.01424186046511628,
 None,
 None,
 None,
 None,
 0.0129,
 None]

In [232]:
indexes = []
for i in issues.keys():
    for j in candidates.keys():
#         print(str(i+'&'+j))
        a = str(i+' & '+j)
        indexes.append(a)
indexes
total_score_df = pd.DataFrame({'Sentiment Score (if lift > 1)':total_score}, index = indexes )
total_score_df

,Sentiment Score (if lift > 1)
healthcare & beto,0.007121
healthcare & cruz,0.014242
job & beto,NaN
job & cruz,NaN
crime & beto,NaN
crime & cruz,NaN
gun & beto,0.012900
gun & cruz,NaN


In [233]:
final_chart = pd.concat([lift_chart,total_score_df],axis=1)
final_chart

,Lift,Sentiment Score (if lift > 1)
healthcare & beto,1.5637,0.007121
healthcare & cruz,1.3201,0.014242
job & beto,0.0000,NaN
job & cruz,0.0000,NaN
crime & beto,0.0000,NaN
crime & cruz,0.0000,NaN
gun & beto,1.0546,0.012900
gun & cruz,0.4452,NaN
